### OCR a tabled/ columned data

In [2]:
import pytesseract
from PIL import Image

In [3]:
img = Image.open("data/index_02.JPG")

In [5]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
ocr_image = pytesseract.image_to_string(img)
print(ocr_image)   # the structure is not maintained, lot of noise, what if we need ner? preprocessing helps

INDEX NOMINUM.

SCRIPSIT KAROLUS HAMPE.

Maior numerus indicat paginam, minor lineam.

A.v. E.

Aaron 38, 20. 123, 18. 168, 6. 203, 38. |

Abacue propheta 36, 5. 319,29. 373, 23.
Abagarus rex Edessenorum 382, 13.
Abdenago 585, 3.

Abel 26, 11.

Abigail 606, 6.

Abraham 26,18. 124,18. 133, 23, 2:
264. 302, 13. 303, 13. 320, 40. 328, 1.
496, 22.

Absalon 102, 5.

Acgfredi v. Ecgfridus.

Achab rex 504, 1.

Achaz rex 504,1.

Achiel 606, 19.

Achilon 496, 26.

Adal-, Adel-, Adhel-, Aedel-, Aedel-,
‘Aedil-, Aed
Athel-, Athil-, Aethel-, Aethil-
Ethel-.

Aedelbaldusabb.S. Petri Wiremuthen-
sis et S. Pauli Girwensis 110, 28.

Aedilberctus, Aedilberthus ep. Ha-
gulstadensis, antea ep. Candidae
casae sive Witernensis 27,43. 72,16.

Adalbertus, Aedilberctus sive Magus,
discipulus et capellanus Arnonis
archiep. Salisburg., postea_abb.
Ferrariensis 254, 1. 320, 41. 322, 19.
416. 418, 14. 422, 10.

Adalpertabb. [Tegernseensis ?]497,2.

Adalbertus presb. monasterii 8. Mar-
tini Turonensis 399, 3.

Aed

We need to find out columns first and then perform OCR to better get the named entities. For that we need boundary boxes around the columns. We can preprocess to achieve it using opencv.

1. Gray scale the image
2. Blur image (to identify overall structure, and not focusing on text itself) 
2. Create threshold (and kernal) to separate text block 
3.  Perform dilation (~white thickening)
4. Perform contour (finding boundaries)  
5. Perform loop to only draw boundrary box of specific size (to exclude small bbox)



In [38]:
import cv2
import numpy as np


In [39]:
img = cv2.imread("data/index_02.JPG")
base_image = img.copy()

### To get the structure of the image 

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("temp/index_02_img_gray.jpg", img_gray)

True

In [ ]:
img_blur = cv2.GaussianBlur(img_gray, (7, 7), 0)
cv2.imwrite("temp/index_02_img_blur.jpg", img_blur)

True

In [ ]:
thresh, img_thresh = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imwrite("temp/index_02_img_thresh.jpg", img_thresh)

True

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 13))
img_dilate = cv2.dilate(img_thresh, kernel, iterations=1)  # thins the text as we didnt invert the img
cv2.imwrite("temp/index_02_img_dilate.jpg", img_dilate)

True

In [ ]:
# Grab the content of the structure using contours
contours = cv2.findContours(img_dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = contours[0] if len(contours) == 2 else contours[1]

In [45]:
print(len(contours)), print(type(cnts))

2
<class 'tuple'>


(None, None)

In [ ]:
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [ ]:
results  = []
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:  # we are having this condition to make sure the small bboxes are eliminated and only big columns are extracted
        roi = img[y:y + h, x:x + w]
        cv2.imwrite("temp/index_roi.png", roi)
        # draw bbox
        cv2.rectangle(img, (x, y), (x+w, y+h), (36, 255, 12), 2)  # (36, 255, 12) - green color
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")  # for further post processing
        for item in ocr_result:
            results.append(item)
cv2.imwrite("temp/index_02_bbox_new.png", img) # save the img

True

In [49]:
print (results)

['A. v. E.', '', 'Aaron 38, 20. 123, 18. 168, 6. 203, 38.', 'Abacuc propheta 36,5. 319,29. 373, 23.', 'Abagarus rex Edessenorum 382, 13.', 'Abdenago 585, 3.', 'Abel 26, 11.', 'Abigail 606, 6.', 'Abraham 26,18, 124,18. 133, 23.', '264. 302, 13. 303, 13. 320, 40.', '496, 22.', 'Absalon 102, 5.', 'Acgfredi v. Ecgtridus.', 'Achab rex 504, 1.', 'Achaz rex 504, 1.', 'Achiel 606, 19.', 'Achilon 496, 26.', 'Adal-, Adel-, Adhel-, Aedel-, Aed', '', 'Aedil-, Aedil- Edel-, Edil-, Athal-,', 'Athel-, Athil-, Aethel-, Aethil-,', 'Ethel-.', 'Aedelbaldusabb.S. Petri Wiremuthen-', 'sis et S. Pauli Girwensis 110, 28.', 'Aedilberctus, Aedilberthus ep. Ha-', 'gulstadensis, antea ep. Candidae', 'casae sive Witernensis 27,43. 72,16.', 'Adalbertus, Aedilberctus sive Magus,', 'discipulus et capellanus Arnonis', 'archiep. Salisburg., postea_abb.', 'Ferrariensis 254, 1. 320, 41. 322, 19.', '416. 418, 14. 422, 10.', 'Adalpertabb.[Tegernseensis ?]497,23.', 'Adalbertus presb. monasterii S. Mar-', 'tini Turonensis 3

In [50]:
entities = []
for item in results:
    item = item.strip().replace("\n", "")
    item = item.split(" ")[0]
    if len(item) > 2:
        if item[0] == "A" and "-" not in item:
            item = item.split(".")[0].replace(",", "").replace(";", "")
            entities.append(item)

In [51]:
print(entities)

['Aaron', 'Abacuc', 'Abagarus', 'Abdenago', 'Abel', 'Abigail', 'Abraham', 'Absalon', 'Acgfredi', 'Achab', 'Achaz', 'Achiel', 'Achilon', 'Aedelbaldusabb', 'Aedilberctus', 'Adalbertus', 'Adalpertabb', 'Adalbertus', 'Aedilburga', 'Aethelfleda', 'Adalgarius', 'Adelgisa', 'Adalgisus', 'Aethelhardus', 'Aedelhardus', 'Adalhardus', 'Adalmondus', 'Adelperga', 'Adalpert', 'Aedelredus', 'Aethilredus', 'Aedilthyda', 'Adhelricus', 'Aedilthyda', 'Aedelwaldus', 'Aedilthyda', 'Adalwinus', 'Aethiluuinus', 'Adam', 'Adam', 'Adaula', 'Adoredus', 'Adrianus', 'Adriaticum', 'Aegesippus', 'Aegil', 'Aegyptus', 'Aegyptii', 'Aegyptiacascola', 'Aeine', 'Aelberhtus', 'Aelfvaldus', 'Aelim', 'Aeneades', 'Aesculapius', 'Aesne', 'Aethiopia', 'Africa', 'Africanus', 'Agar', 'Agareni', 'Agathensis', 'Aginus']


In [52]:
entities = list(set(entities))  # remove duplicates
print (entities)

['Adelgisa', 'Agareni', 'Aegyptii', 'Aedelhardus', 'Aedilthyda', 'Aegyptus', 'Adalgarius', 'Agathensis', 'Adalmondus', 'Adalbertus', 'Abagarus', 'Agar', 'Aethelfleda', 'Adalgisus', 'Achaz', 'Abigail', 'Aedelbaldusabb', 'Aethiluuinus', 'Abraham', 'Aaron', 'Adalwinus', 'Adaula', 'Aethelhardus', 'Adhelricus', 'Acgfredi', 'Adalpert', 'Adoredus', 'Aelberhtus', 'Adam', 'Aegesippus', 'Africanus', 'Abel', 'Aesculapius', 'Aegyptiacascola', 'Aedelwaldus', 'Achiel', 'Adalpertabb', 'Aelim', 'Aeneades', 'Achilon', 'Aesne', 'Aedelredus', 'Aedilburga', 'Absalon', 'Aeine', 'Abacuc', 'Aelfvaldus', 'Aethiopia', 'Aginus', 'Adelperga', 'Adrianus', 'Aedilberctus', 'Aegil', 'Africa', 'Aethilredus', 'Abdenago', 'Achab', 'Adalhardus', 'Adriaticum']


In [53]:
entities.sort()

In [54]:
print(entities)

['Aaron', 'Abacuc', 'Abagarus', 'Abdenago', 'Abel', 'Abigail', 'Abraham', 'Absalon', 'Acgfredi', 'Achab', 'Achaz', 'Achiel', 'Achilon', 'Adalbertus', 'Adalgarius', 'Adalgisus', 'Adalhardus', 'Adalmondus', 'Adalpert', 'Adalpertabb', 'Adalwinus', 'Adam', 'Adaula', 'Adelgisa', 'Adelperga', 'Adhelricus', 'Adoredus', 'Adrianus', 'Adriaticum', 'Aedelbaldusabb', 'Aedelhardus', 'Aedelredus', 'Aedelwaldus', 'Aedilberctus', 'Aedilburga', 'Aedilthyda', 'Aegesippus', 'Aegil', 'Aegyptiacascola', 'Aegyptii', 'Aegyptus', 'Aeine', 'Aelberhtus', 'Aelfvaldus', 'Aelim', 'Aeneades', 'Aesculapius', 'Aesne', 'Aethelfleda', 'Aethelhardus', 'Aethilredus', 'Aethiluuinus', 'Aethiopia', 'Africa', 'Africanus', 'Agar', 'Agareni', 'Agathensis', 'Aginus']
